In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

## 2D meshing example

This notebook shows how to

1. Load and visualize a volume
2. Select a plane from a volume
3. Apply image filters to and segment image
4. Generate a 2D mesh from the binary image
5. Visualize and save the mesh to gmsh22 format

In [ ]:
from nanomesh.volume import Volume
from nanomesh.plane import Plane
import pyvista as pv
from skimage import filters
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
from skimage.color import rgb2gray

In [ ]:
banner_o = io.imread(r'banner_o.png')
plane = Plane(rgb2gray(banner_o))
plane.show()

In [ ]:
thresh = 0.5
seg = plane.apply(np.digitize, bins=[thresh])
seg.show()

In [ ]:
from nanomesh.mesh2d import Mesher2D

mesher = Mesher2D(seg.image)

mesher.add_points(label=0, method='kmeans', point_density=0.0018, verbose=True)
mesher.add_points(label=1, method='gmm', point_density=0.0018)
mesher.generate_contours(max_contour_dist=10, contour_precision=3)
mesher.generate_edge_contours(max_contour_dist=30)
mesher.generate_mesh()
mesher.generate_domain_mask()

mesher.plot_steps()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'jet'

banner_no_o = io.imread(r'banner_no_o.png')

image = mesher.image
points = mesher.surface_mesh.vertices
triangles = mesher.surface_mesh.faces
labels = mesher.labels

x, y = points.T[::-1]

fig, ax = plt.subplots(figsize=(8, 2))
fig.tight_layout(pad=0)

ax.imshow(banner_no_o)
ax.axis('off')
ax.margins(0)

colors = np.arange(len(triangles))
mask0 = (labels == 0)
ax.tripcolor(x, y, triangles=triangles, mask=mask0, facecolors=colors)
ax.triplot(x, y, triangles=triangles, mask=mask0, color='black', lw=0.5)

mask1 = (labels == 1)
ax.triplot(x, y, triangles=triangles, mask=mask1, lw=0.5, alpha=0.8)
